## Traditional Machine Learning Model

Use SVM, Random forest to predict PVC

Note: This code have to be run after preprocessing.ipynb

In [82]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import matplotlib.pyplot as plt
%matplotlib inline
import pickle


In [8]:
trainset = pickle.load(open("./trainset.p",'rb'))
testset = pickle.load(open("./testset.p",'rb'))

In [44]:
trainset.head()

,R_interval_mean,R_interval_sd,age,annotation,gender,id,lead,p2p_mean,p2p_std,peak,strip,stripPVC
0,227.800000,48.945139,64,symbol 67 + 225 V 487 ...,0,200,V1,942.583454,1695.116429,"[488, 1434, 2332]",0 0 0.064045 1 0.071534 2...,True
1,231.000000,48.446534,64,symbol 3611 + 3738 N 3975 ...,0,200,V1,942.583454,1695.116429,"[4647, 5383, 5607, 5822, 6549]",0 3600 0.065740 3601 0.062907 3...,True
2,262.538462,91.607537,64,symbol 7260 V 7455 V 7899 ...,0,200,V1,942.583454,1695.116429,"[7456, 7900, 8395, 8726, 8727, 8728, 9171, 996...",0 7200 0.032907 7201 0.05067...,True
3,266.076923,84.273786,64,symbol 10901 N 11132 N 11361 ...,0,200,V1,942.583454,1695.116429,"[10902, 11618, 11858, 12723, 13053, 13523, 141...",0 10800 -0.110721 10801 -0.03604...,True
4,213.187500,85.949999,64,symbol 14449 + 14538 V 14903 ...,0,200,V1,942.583454,1695.116429,[14904],0 14400 -0.150412 14401 -0.12883...,True


In [45]:
testset.head()

,R_interval_mean,R_interval_sd,age,annotation,gender,id,lead,p2p_mean,p2p_std,peak,strip,stripPVC
0,212.625000,68.564636,23,symbol 9 + 46 F 209 ...,1,208,V1,335.860465,329.396434,"[46, 209, 484, 699, 1182, 1378, 1580, 1861, 20...",0 0 0.250416 1 0.252630 2...,True
1,209.588235,55.693551,23,symbol 3606 V 3886 N 4097 ...,1,208,V1,335.860465,329.396434,"[3606, 3886, 4099, 4565, 4758, 4760, 4945, 519...",0 3600 -2.944454 3601 -3.007464 3...,True
2,181.157895,34.838368,23,symbol 7349 N 7547 F 7750 ...,1,208,V1,335.860465,329.396434,"[7350, 7548, 8147, 8317, 8531, 8726, 8923, 105...",0 7200 0.426845 7201 0.43757...,True
3,182.111111,33.520953,23,symbol 10973 N 11155 N 11324 ...,1,208,V1,335.860465,329.396434,"[10974, 11155, 11324, 11527, 11698, 11888, 120...",0 10800 -0.470647 10801 -0.35374...,True
4,203.941176,19.627324,23,symbol 14449 N 14619 V 14845 ...,1,208,V1,335.860465,329.396434,"[14449, 14623, 14845, 15036, 15223, 15235, 154...",0 14400 0.358137 14401 0.20314...,True


Normalization

In [58]:
from sklearn import preprocessing

col = trainset.drop(['peak','annotation','strip','stripPVC','id','lead'],axis=1).columns.values
scale = preprocessing.StandardScaler()
scale.fit(trainset.drop(['peak','annotation','strip','stripPVC','id','lead'], axis=1))
n_trainset = pd.DataFrame(scale.transform(trainset.drop(['peak','annotation','strip','stripPVC','id','lead'], axis=1)),columns=col)
n_testset = pd.DataFrame(scale.transform(testset.drop(['peak','annotation','strip','stripPVC','id','lead'], axis=1)),columns=col)

Calculate Baseline

In [59]:
len(trainset[trainset.stripPVC==True])/len(trainset)

0.5020949720670391

### Build models

In [73]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score

In [76]:
def metric(pred, real):
    ConfusionMatrix= pd.DataFrame(confusion_matrix(pred,real), columns=['True','False'], index=['True','False'])
    print(ConfusionMatrix)
    print('----------------')
    print('Precision:', ConfusionMatrix.iloc[0,0] / (ConfusionMatrix.iloc[0,0] + ConfusionMatrix.iloc[1,0]))
    print('Recall:', ConfusionMatrix.iloc[0,0] / (ConfusionMatrix.iloc[0,0] + ConfusionMatrix.iloc[1,1]))
    print('Accuracy:', (ConfusionMatrix.iloc[0,0] + ConfusionMatrix.iloc[1,1]) / (ConfusionMatrix.iloc[0,0] + ConfusionMatrix.iloc[1,0] + ConfusionMatrix.iloc[0,1] + ConfusionMatrix.iloc[1,1]))
    print('AUC:', roc_auc_score(pred,real))

#### Random forest

In [61]:
clf = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)
clf.fit(n_trainset , trainset['stripPVC'])
pred = clf.predict(n_testset)

In [77]:
metric(pred,testset['stripPVC'])

       True  False
True    387     69
False   516    818
----------------
Precision: 0.42857142857142855
Recall: 0.3211618257261411
Accuracy: 0.6731843575418994
AUC: 0.7309388069123334



#### SVM

In [78]:
clf_svm = SVC(kernel='rbf')
clf_svm.fit(n_trainset, trainset['stripPVC'])
svm_pred = clf_svm.predict(n_testset)

In [79]:
metric(svm_pred ,testset['stripPVC'])

       True  False
True    586    200
False   317    687
----------------
Precision: 0.6489479512735327
Recall: 0.4603299293008641
Accuracy: 0.711173184357542
AUC: 0.7149050109992598
